In [ ]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

##Putting It Together
Now that the basic components of the Pipeline API are covered, let's construct a pipeline that you might want to use in an algorithm.

To start, let's first create a filter to narrow down the types of securities coming out of the pipeline. In this example, you will create a filter to select for securities that meet the following criteria:
- It's a primary share
- It's listed as a common stock
- It's not a [depositary receipt](http://www.investopedia.com/terms/d/depositaryreceipt.asp) (ADR/GDR)
- It's not trading [over-the-counter](http://www.investopedia.com/terms/o/otc.asp) (OTC)
- It's not [when-issued](http://www.investopedia.com/terms/w/wi.asp) (WI)
- Doesn't have a name indicating it's a [limited partnership](http://www.investopedia.com/terms/l/limitedpartnership.asp) (LP)
- Doesn't have a company reference entry indicating it's a LP
- It's not an [ETF](http://www.investopedia.com/terms/e/etf.asp) (has Morningstar fundamental data)


####Why These Criteria?
Selecting for primary shares and common stock helps to select only a single security for each company. In general, primary shares are a good representative asset of a company so these are selected in our pipeline.

ADRs and GDRs are issuances in the US equity market for stocks that trade on other exchanges. Frequently, there is inherent risk associated with depositary receipts due to currency fluctuations so you can exclude them from the pipeline.

OTC, WI, and LP securities are not tradeable with most brokers. As a result, you exclude them from the pipeline.

###Creating the Pipeline
Let's create a filter for each criterion and combine them together to create a `tradeable_stocks` filter. First, you need to import the Morningstar `DataSet` as well as the `IsPrimaryShare` builtin filter.

In [ ]:
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.filters.fundamentals import IsPrimaryShare

Now you can define the filters:

In [ ]:
# Filter for primary share equities. IsPrimaryShare is a built-in filter.
primary_share = IsPrimaryShare()

# Equities listed as common stock (as opposed to, say, preferred stock).
# 'ST00000001' indicates common stock.
common_stock = Fundamentals.security_type.latest.eq('ST00000001')

# Non-depositary receipts. Recall that the ~ operator inverts filters,
# turning Trues into Falses and vice versa
not_depositary = ~Fundamentals.is_depositary_receipt.latest

# Equities not trading over-the-counter.
not_otc = ~Fundamentals.exchange_id.latest.startswith('OTC')

# Not when-issued equities.
not_wi = ~Fundamentals.symbol.latest.endswith('.WI')

# Equities without LP in their name, .matches does a match using a regular
# expression
not_lp_name = ~Fundamentals.standard_name.latest.matches('.* L[. ]?P.?$')

# Equities with a null value in the limited_partnership Morningstar
# fundamental field.
not_lp_balance_sheet = Fundamentals.limited_partnership.latest.isnull()

# Equities whose most recent Morningstar market cap is not null have
# fundamental data and therefore are not ETFs.
have_market_cap = Fundamentals.market_cap.latest.notnull()

# Filter for stocks that pass all of our previous filters.
tradeable_stocks = (
    primary_share
    & common_stock
    & not_depositary
    & not_otc
    & not_wi
    & not_lp_name
    & not_lp_balance_sheet
    & have_market_cap
)

Note that when defining the filters, several `Classifier` methods are used that you haven't yet seen including `notnull`, `startswith`, `endswith`, and `matches`. Documentation on these methods are available [here](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier).

Next, let's create a filter for the top 30% of tradeable stocks by 20-day average dollar volume. You can call this the `base_universe`.

In [ ]:
base_universe = AverageDollarVolume(window_length=20, mask=tradeable_stocks).percentile_between(70, 100)

####Built-in Base Universe

You have just defined the base universe to select 'tradeable' securities with high dollar volume. However, Quantopian has several built-in filters that do something similar, the best and newest of which is the [QTradableStocksUS](https://www.quantopian.com/help#quantopian_pipeline_filters_QTradableStocksUS). The QTradableStocksUS is a built-in pipeline filter that selects a daily universe of stocks that are filtered in three passes and adhere to a set of criteria to yield the most liquid universe possible without any size constraints. The QTradableStocksUS therefore has no size cutoff unlike its predecessors, the [Q500US](https://www.quantopian.com/help#quantopian_pipeline_filters_Q500US) and the [Q1500US](https://www.quantopian.com/help#quantopian_pipeline_filters_Q1500US). More detail on the selection criteria of the QTradableStocksUS can be found [here](https://www.quantopian.com/posts/working-on-our-best-universe-yet-qtradablestocksus).

To simplify the pipeline, let's replace what you've already written for the `base_universe` with the `QTradableStocksUS` built-in filter. First, you need to import it.

In [ ]:
from quantopian.pipeline.filters import QTradableStocksUS

Then, let's set the base_universe to the `QTradableStocksUS`.

In [ ]:
base_universe = QTradableStocksUS()

Now that you have a filter `base_universe` that you can use to select a subset of securities, let's focus on creating factors for this subset. For this example, let's create a pipeline for a mean reversion strategy. In this strategy, you can look at the 10-day and 30-day moving averages (close price). Let's plan to open equally weighted long positions in the 75 securities with the least (most negative) percent difference and equally weighted short positions in the 75 with the greatest percent difference. To do this, let's create two moving average factors using the `base_universe` filter as a mask. Then let's combine them into a factor computing the percent difference.

In [ ]:
# 10-day close price average.
mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10, mask=base_universe)

# 30-day close price average.
mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30, mask=base_universe)

percent_difference = (mean_10 - mean_30) / mean_30

Next, let's create filters for the top 75 and bottom 75 equities by `percent_difference`.

In [ ]:
# Creating a filter to select securities to short.
shorts = percent_difference.top(75)

# Creating a filter to select securities to long.
longs = percent_difference.bottom(75)

Let's then combine `shorts` and `longs` to create a new filter that you can use as the screen of the pipeline:

In [ ]:
securities_to_trade = (shorts | longs)

Since earlier filters were used as masks, in this final filter, when you use `securities_to_trade` as a screen, the output securities will meet the criteria outlined at the beginning of the lesson (primary shares, non-ETFs, etc.). They will also have high dollar volume.

Finally, let's instantiate the pipeline. Since you are planning on opening equally weighted long and short positions later, the only information that you actually need from the pipeline is which securities you want to trade (the pipeline index) and whether or not to open a long or a short position. Let's add the `longs` and `shorts` filters to the pipeline and set the screen to be `securities_to_trade`.

In [ ]:
def make_pipeline():
    
    # Base universe filter.
    base_universe = QTradableStocksUS()
    
    # 10-day close price average.
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10, mask=base_universe)

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30, mask=base_universe)

    # Percent difference factor.
    percent_difference = (mean_10 - mean_30) / mean_30
    
    # Creating a filter to select securities to short.
    shorts = percent_difference.top(75)

    # Creating a filter to select securities to long.
    longs = percent_difference.bottom(75)
    
    # Filter for the securities that you want to trade.
    securities_to_trade = (shorts | longs)
    
    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts
        },
        screen=securities_to_trade
    )

Running this pipeline will result in a DataFrame containing 2 columns. Each day, the columns will contain boolean values that you can use to decide whether you want to open a long or short position in each security.

In [ ]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

longs shorts
2015-05-05 00:00:00+00:00 Equity(39 [DDC])    False   True
                          Equity(351 [AMD])    True  False
                          Equity(371 [TVTY])   True  False
                          Equity(474 [APOG])  False   True
                          Equity(523 [AAN])   False   True